**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
suppressMessages(suppressWarnings(library(pheatmap)))
show_env()

You are in Singularity: singularity_proj_combeffect 
BASE DIRECTORY:     /data/reddylab/Kuei 
WORK DIRECTORY:     /data/reddylab/Kuei/out 
CODE DIRECTORY:     /data/reddylab/Kuei/code 
PATH OF SOURCE:     /data/reddylab/Kuei/source 
PATH OF EXECUTABLE: /data/reddylab/Kuei/bin 
PATH OF ANNOTATION: /data/reddylab/Kuei/annotation 
PATH OF PROJECT:    /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS:    /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 


## Import log2FC

In [2]:
ASSAY    = "Tewhey_K562_TileMPRA"
FOLDER   = "coverage_astarrseq_peak_macs_input"
DATASETS = c("OL13", "OL43", "OL45")
TYPE     = "raw"

In [3]:
###
lst = lapply(DATASETS, function(DATASET){
    fdiry = file.path(FD_RES, "results", ASSAY, FOLDER, "summary")
    fname = paste("result", "Log2FC", "raw", "deseq", DATASET, "tsv", sep=".")
    fpath = file.path(fdiry, fname)
    dat   = read_tsv(fpath, show_col_types = FALSE)
    dat   = dat %>% dplyr::mutate(Dataset = DATASET)
    return(dat)
})

###
dat = bind_rows(lst)

###
dat_log2fc = dat
print(dim(dat))
head(dat)

[1] 1451    8


Peak,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Dataset
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
chr11_61792068_61793464,260242.12,1.6665705,0.04117109,40.479145,0.000000e+00,0.000000e+00,OL13
chr11_61800085_61801113,216437.58,1.8554055,0.04972890,37.310405,1.112851e-291,6.120682e-293,OL13
chr11_61806630_61807154,49327.33,-1.3562455,0.08825841,-15.366758,2.735191e-53,4.628784e-53,OL13
chr11_61814735_61817343,682819.52,2.5310135,0.03452518,73.309201,0.000000e+00,0.000000e+00,OL13
chr11_61822094_61822443,20363.10,-1.7731145,0.14597377,-12.146802,5.965339e-34,7.719850e-34,OL13
chr11_61825795_61826306,29539.40,-0.3921839,0.07941651,-4.938318,7.879932e-07,7.879932e-07,OL13


In [4]:
fdiry = file.path(FD_RES, "results", "comparison")
fname = "dat_peaks_info.tsv"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath, show_col_types = FALSE)

###
dat_peak_info = dat
dat_peak      = dat %>% dplyr::select(Peak, Dataset, Region)
print(dim(dat))
head(dat)

[1] 1451    6


Peak,Chrom,Start,End,Dataset,Region
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
chr11_61792068_61793464,chr11,61792068,61793464,OL13,FADS
chr11_61800085_61801113,chr11,61800085,61801113,OL13,FADS
chr11_61806630_61807154,chr11,61806630,61807154,OL13,FADS
chr11_61814735_61817343,chr11,61814735,61817343,OL13,FADS
chr11_61822094_61822443,chr11,61822094,61822443,OL13,FADS
chr11_61825795_61826306,chr11,61825795,61826306,OL13,FADS


## Rank norm

In [5]:
fun_ranknorm = function(x){
    return(rank(x)/length(x))
}

In [6]:
dat = left_join(dat_peak, dat_log2fc, by=c("Peak", "Dataset"))

dat = dat %>%
    dplyr::group_by(Region) %>%
    dplyr::mutate(RankNorm_Region  = fun_ranknorm(log2FoldChange)) %>%
    dplyr::ungroup()

dat = dat %>%
    dplyr::group_by(Dataset) %>%
    dplyr::mutate(RankNorm_Dataset = fun_ranknorm(log2FoldChange)) %>%
    dplyr::ungroup()

dat = dat %>%
    dplyr::mutate(RankNorm_Total  = fun_ranknorm(log2FoldChange))

dat_ranknorm = dat
print(dim(dat))
tail(dat)

[1] 1451   12


Peak,Dataset,Region,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,RankNorm_Region,RankNorm_Dataset,RankNorm_Total
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr2_61524551_61524988,OL45,BCL11A,2088.957,-1.3009654,0.06373091,-20.413414,1.270903e-92,4.714638e-92,0.01630435,0.01352657,0.0241213
chr2_61529854_61531204,OL45,BCL11A,8875.031,0.2766738,0.05305414,5.214934,1.838829e-07,2.271107e-07,0.70652174,0.58067633,0.5789111
chr2_61536312_61537087,OL45,BCL11A,8311.507,2.1228528,0.04971160,42.703367,0.000000e+00,0.000000e+00,0.93478261,0.89758454,0.8904204
chr2_61537852_61539255,OL45,BCL11A,84176.640,4.2671644,0.04616342,92.436047,0.000000e+00,0.000000e+00,0.98913043,0.98937198,0.9917298
chr2_61545554_61545869,OL45,BCL11A,2414.730,-0.9562369,0.06059907,-15.779730,4.290224e-56,1.083020e-55,0.15217391,0.12173913,0.1364576
chr2_61549710_61550633,OL45,BCL11A,4649.512,-0.8560994,0.05269804,-16.245374,2.408672e-59,6.327348e-59,0.23913043,0.16714976,0.1819435


In [7]:
fdiry = file.path(FD_RES, "results", "comparison", "comparison_global")
fname = "result.ranknorm.tmpra.tsv"
fpath = file.path(fdiry, fname)

dat = dat_ranknorm
write_tsv(dat, fpath)